In [ ]:
!pip -q install --upgrade openai pandas openpyxl tqdm

import os, io, json, time, math
import numpy as np
import pandas as pd
from tqdm import tqdm
from google.colab import files
from openai import OpenAI

In [ ]:
assert os.environ.get("OPENAI_API_KEY"), "환경변수 OPENAI_API_KEY를 먼저 설정하세요."
client = OpenAI()

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[filename]))

In [ ]:
MODEL = "gpt-4o"
TEMPERATURE = 0
N_RUNS = 3
MAX_EMOTIONS = 5

In [ ]:
SYSTEM_INSTRUCTION = (
    "당신은 감성분석 전문가입니다. "
    "입력 문장에서 느껴지는 감정들을 한국어로 명명하고, 각 감정의 강도를 0~1 사이 확률로 산출하세요. "
    f"감정 라벨은 자유롭게 정하되, 최대 {MAX_EMOTIONS}개까지만 선택하세요. "
    "모든 확률의 합은 정확히 1.0이 되도록 하세요. "
    "반드시 순수 JSON만 출력하고, 형식은 다음과 같습니다: "
    "{\"감정라벨\": 확률, ...} (예: {\"억울함\": 0.4, \"분노\": 0.3, \"불안\": 0.3}). "
    "설명, 코드블록, 불릿 등 JSON 외의 텍스트는 포함하지 마세요." )

USER_TEMPLATE = """다음 문장의 감정 확률 분포를 산출하세요.

문장:
\"\"\"{text}\"\"\"

요구사항 요약:
- 감정 라벨: 자유롭게(한국어, 구체적/자연스러운 단어; 예: 억울함, 외로움, 분노, 불안, 수치심, 무력감 등)
- 감정 개수 최대 {max_k}개
- 확률 합 = 1.0
- 출력은 순수 JSON만 (예: {{"억울함": 0.4, "분노": 0.3, "불안": 0.3}})
"""

def _normalize_probs(d: dict) -> dict:
    if not isinstance(d, dict) or not d:
        return {}
    clean = {}
    for k, v in d.items():
        try:
            p = float(v)
        except:
            p = 0.0
        if not np.isfinite(p) or p < 0:
            p = 0.0
        clean[str(k).strip()] = p
    s = sum(clean.values())
    if s <= 0:
        return {}
    return {k: (v / s) for k, v in clean.items()}

def _topk(d: dict, k: int) -> dict:
    if not d:
        return {}
    items = sorted(d.items(), key=lambda x: x[1], reverse=True)[:k]
    return _normalize_probs(dict(items))

def _call_once(text: str, max_retries: int = 3, backoff: float = 1.5) -> dict:
    prompt = USER_TEMPLATE.format(text=text, max_k=MAX_EMOTIONS)
    for attempt in range(max_retries):
        try:
            resp = client.responses.create(
                model=MODEL,
                temperature=TEMPERATURE,
                instructions=SYSTEM_INSTRUCTION,
                input=prompt,
            )
            raw = resp.output_text.strip()
            data = json.loads(raw)
            if not isinstance(data, dict):
                raise ValueError("JSON 객체가 아님")
            data = _normalize_probs(data)
            data = _topk(data, MAX_EMOTIONS)
            return data
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"⚠️ 분석 실패(최종): {e}")
                return {}
            time.sleep(backoff ** attempt)

def analyze_emotion_distribution_3run(text: str) -> dict:
    if not isinstance(text, str) or not text.strip():
        return {}
    agg = {}
    for _ in range(N_RUNS):
        d = _call_once(text)
        for k, v in d.items():
            agg[k] = agg.get(k, 0.0) + v
    if not agg:
        return {}
    for k in list(agg.keys()):
        agg[k] /= N_RUNS
    return _normalize_probs(agg)

def extract_top_emotions(em_dict, top_n=3):
    if not isinstance(em_dict, dict) or not em_dict:
        return [None] * top_n
    items = sorted(em_dict.items(), key=lambda x: x[1], reverse=True)[:top_n]
    names = [k for k, _ in items]
    return names + [None] * (top_n - len(names))

In [ ]:
if "원자료" not in df.columns:
    raise ValueError("엑셀에 '원자료' 컬럼이 필요합니다.")

texts = df["원자료"].astype(str).fillna("").tolist()

out = []
for t in tqdm(texts, desc="감정 분석 중"):
    out.append(analyze_emotion_distribution_3run(t))

df["감정확률분포"] = out
tops = df["감정확률분포"].apply(lambda d: pd.Series(extract_top_emotions(d, top_n=3), index=["감정1","감정2","감정3"]))
df = pd.concat([df, tops], axis=1)

output_file = "/content/gpt_감정확률분석_결과.xlsx"
df.to_excel(output_file, index=False)
files.download(output_file)

print("완료 ✅ →", output_file)